Welcome to the mobility notebook. This notebook does an in-depth analysis of the mobility of HPVs for carrying water. It uses the mobility_module.py package, this is where all of the models are stored.



In [38]:
import pandas as pd
import mobility_module as mm
import plotly.express as px

In [39]:
######################
#### Import Data #####
with open("../data/mobility-model-parameters.csv") as csv_file:
    # read the csv file
    allHPV_param_df = pd.read_csv("../data/mobility-model-parameters.csv")

mo = mm.model_options()

Below, choose which model to use, 1 = Sprott, 2 = Cycling, 3 = walking 

In [40]:
mo.model_selection = 2

Below, sets up which HPVs to include given the input model

In [41]:

####################
##### Options ######
####################

if mo.model_selection == 1:
    # sprott, include all
    filter_col = -1
    filter_value = 1

elif mo.model_selection == 2:
    # cycling:
    filter_col = 5
    filter_value = 1

if mo.model_selection > 2:
    # walking only
    filter_col = 6
    filter_value = 0



In [42]:

# selectHPVs you're interested in
if filter_col > 0:
    column_names = [
        "Name",
        "LoadLimit",
        "AverageSpeedWithoutLoad",
        "Drive",
        "GroundContact",
        "Pilot",
        "Crr",
        "Efficiency",
        "Weight",
    ]  # copied from
    col = column_names[filter_col]
    param_df = allHPV_param_df.loc[(allHPV_param_df[col] == filter_value)]
else:
    param_df = allHPV_param_df


In [43]:

# initialise variables, options, MET, hpv and results classes, populated with data via 'init' functions
mv = mm.model_variables()
met = mm.MET_values(mv)
hpv = mm.HPV_variables(param_df, mv)
mr = mm.model_results(hpv, mo)


In [44]:
# change crr for different scenarios
hpv.Crr[0][0][0] = 0.004

In [45]:
####### SPROTT MODEL ########
if mo.model_selection == 1:
    mr.v_load_matrix3d, mr.load_matrix3d = mm.mobility_models.sprott_model(
        hpv, mv, mo, mr
    )

####### Numerical MODEL ########
elif mo.model_selection > 1:
    mr.v_load_matrix3d, mr.load_matrix3d = mm.mobility_models.numerical_mobility_model(
        mr, mv, mo, met, hpv
    )




In [46]:

####### Organise Results #######
mo.slope_scene = 0
mr.load_results(hpv, mv, mo)


In [47]:

# mr.velocitykgs



mm.plotting_hpv.load_plot_plotly(mr, mo, hpv)
mm.plotting_hpv.bar_plot_loading_distance(mr, mo, hpv, mv)
mm.plotting_hpv.slope_plot_plotly(mr, mo, hpv)



In [48]:
slope_name = mr.slope_vector_deg.flat[mo.slope_scene]
chart_title = "Efficiency at %0.2f degrees, with model %s" % (
    slope_name,
    mr.model_name,
)

df = mr.create_dataframe_single_scenario(hpv, mv, mo.load_scene, mo.slope_scene)

df.to_csv("efficiency.csv")

In [49]:
# y = mr.v_load_matrix3d[
#     i, :, mo.load_scene
# ]  # SEE ZEROS <-- this is for the minimum weight
# x = mr.slope_matrix3d_deg[i, :, mo.load_scene]
# i += 1

# mr.hpv_name[0][0][0][0] <- bicycle

# bicycle is 
load_v = mr.v_load_matrix3d[
    0, :, -1
]

unload_v = mr.v_unload_matrix3d[
    0, :, -1
]

load_vector= mr.load_matrix3d[
    0, :, -1
]

slope_vector = mr.slope_matrix3d_deg[
    0, :, -1
]


filename_string = f'{mr.hpv_name[0][0][0][0]}_Crr_{hpv.Crr[0][0][0]}'


# create a dataframe with the data
df = pd.DataFrame(
    {
        "slope_vector": slope_vector,
        "load_v": load_v,
        "unload_v": unload_v,
        "load_matrix": load_vector,
    }
)
# setindex slope
df.set_index("slope_vector", inplace=True)

# save the dataframe to a csv
df.to_csv(f'../data/{filename_string}.csv')

